### Week 5 Day 4

AutoGen Core - Distributed

I'm only going to give a Teaser of this!!

Partly because I'm unsure how relevant it is to you. If you'd like me to add more content for this, please do let me know..

In [1]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool
from IPython.display import display, Markdown

from dotenv import load_dotenv

load_dotenv(override=True)

ALL_IN_ONE_WORKER = True

### Start with our Message class

In [2]:

@dataclass
class Message:
    content: str

### And now - a host for our distributed runtime

In [3]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntimeHost

host = GrpcWorkerAgentRuntimeHost(address="localhost:50051")
host.start() 

### Let's reintroduce a tool

In [4]:
# This approach allows to create a tool to search the internet using LangChain and then make it available in AutoGen
serper = GoogleSerperAPIWrapper()
langchain_serper =Tool(name="internet_search", func=serper.run, description="Useful for when you need to search the internet")
autogen_serper = LangChainToolAdapter(langchain_serper)

In [5]:
# Here we have the instructions for two agents explaining pros and cons of using AutoGen in a project
# together with another agent that will judge based on the received motivations

instruction1 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons in favor of choosing AutoGen; the pros of AutoGen."

instruction2 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons against choosing AutoGen; the cons of Autogen."

judge = "You must make a decision on whether to use AutoGen for a project. \
Your research team has come up with the following reasons for and against. \
Based purely on the research from your team, please respond with your decision and brief rationale."

### And make some Agents

In [6]:
#IMPORTANT: we don't really need 2 separate classes for player 1 and 2; we should have used the same class passing the different prompt
# We keep 2 in case we want to use different LLM for player 1 and 2

class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        # Passing the parameter 'name' to super().__init__() means we're initializing the parent (RoutedAgent) with the agent's name.
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Judge(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client)
        
    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        message1 = Message(content=instruction1)
        message2 = Message(content=instruction2)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(message1, inner_1)
        response2 = await self.send_message(message2, inner_2)
        result = f"## Pros of AutoGen:\n{response1.content}\n\n## Cons of AutoGen:\n{response2.content}\n\n"
        judgement = f"{judge}\n{result}Respond with your decision and brief explanation"
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + "\n\n## Decision:\n\n" + response.chat_message.content)


In [ ]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntime

if ALL_IN_ONE_WORKER:
    # Here we have a unique worker (a unique runtime) to run our three agents
    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()

    await Player1Agent.register(worker, "player1", lambda: Player1Agent("player1"))
    await Player2Agent.register(worker, "player2", lambda: Player2Agent("player2"))
    await Judge.register(worker, "judge", lambda: Judge("judge"))

    agent_id = AgentId("judge", "default")

else:
    # This approach will create three different runtimes each one running a single agent
    worker1 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker1.start()
    await Player1Agent.register(worker1, "player1", lambda: Player1Agent("player1"))

    worker2 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker2.start()
    await Player2Agent.register(worker2, "player2", lambda: Player2Agent("player2"))

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()
    await Judge.register(worker, "judge", lambda: Judge("judge"))
    agent_id = AgentId("judge", "default")




In [8]:
response = await worker.send_message(Message(content="Go!"), agent_id)

In [9]:
display(Markdown(response.content))

## Pros of AutoGen:
Here are several pros of using AutoGen for your AI Agent project:

1. **Multi-Agent Communication**: AutoGen supports effective communication and coordination between multiple agents, enabling complex task management and collaboration.

2. **Integration Capabilities**: It can seamlessly integrate with APIs, tools, and external systems, enhancing functionality and allowing for a more interconnected solution.

3. **Custom Role Creation**: AutoGen allows you to define custom roles and behaviors for each agent, providing flexibility and tailoring the agents to meet specific project needs.

4. **Versatility in Applications**: It is particularly well-suited for automation tasks, data analysis, and conversational AI applications, making it a good choice for a variety of use cases.

These advantages can significantly enhance the efficiency and effectiveness of your AI Agent project. 

TERMINATE

## Cons of AutoGen:
Here are some cons associated with using AutoGen for your AI Agent project:

1. **Stability Issues**: Users have reported frequent crashes, software glitches, and bugs that can disrupt productivity.

2. **Documentation Challenges**: The documentation for AutoGen is often considered hard to read and lacks sufficient examples, which can make it difficult for new users to understand and effectively utilize the platform.

3. **Lack of Visual Tools**: AutoGen does not offer a visual builder or no-code editor, which may present a steeper learning curve for those who are not familiar with coding or prefer a more graphical approach to development.

4. **Limited Use Case Suitability**: While AutoGen is suitable for specific applications like LLM experimentation, it may not be as effective for broader, general-purpose AI projects, potentially limiting its versatility.

5. **Complexity for Beginners**: The platform might be better suited for experienced developers, making it less accessible for those who are new to AI development or lack advanced technical skills.

These factors could impact your decision on whether to use AutoGen in your new AI Agent project. 

TERMINATE



## Decision:

Based on the pros and cons presented by the research team, I recommend moving forward with AutoGen for the project.

**Rationale**: The advantages of effective multi-agent communication, integration capabilities, and custom role creation outweigh the concerns regarding stability and documentation challenges. The pros directly align with the project's goals, particularly in enhancing collaboration and flexibility. While stability issues and the complexity for beginners are valid concerns, these can potentially be mitigated through careful planning and a focus on building a strong support structure for the team. Therefore, leveraging AutoGen's strengths seems to present a worthwhile opportunity for our AI Agent project.

TERMINATE

In [10]:
await worker.stop()
if not ALL_IN_ONE_WORKER:
    await worker1.stop()
    await worker2.stop()

In [11]:
await host.stop()